In [104]:
import os
import sys
import glob
from os import write

from tqdm import tqdm
import pandas as pd
import numpy as np
import ast

import plotly as py
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from src.iDrink.iDrinkValPlots import ignore_id_p

sys.path.append(os.path.realpath(os.path.abspath('..')))
from iDrink import iDrinkUtilities

In [105]:
drive = iDrinkUtilities.get_drivepath()

root_iDrink = os.path.join(drive, 'iDrink')
root_val = os.path.join(root_iDrink, "validation_root")
root_stat = os.path.join(root_val, '04_Statistics')
root_omc = os.path.join(root_val, '03_data', 'OMC_new', 'S15133')
root_data = os.path.join(root_val, "03_data")
root_logs = os.path.join(root_val, "05_logs")

dir_stat_cont = os.path.join(root_stat, '01_continuous')
dir_stat_cat = os.path.join(root_stat, '02_categorical')
dir_results = os.path.join(dir_stat_cont, '01_results')
dir_plots_cont = os.path.join(dir_stat_cont, '02_plots')
dir_plots_cat = os.path.join(dir_stat_cat, '02_plots')

csv_val_trials = os.path.join(root_logs, 'validation_trials.csv')
df_val_trials = pd.read_csv(csv_val_trials, sep=';')

csv_settings = os.path.join(root_logs, 'validation_settings.csv')
df_settings = pd.read_csv(csv_settings, sep=';')

csv_calib_error = os.path.join(root_logs, 'calib_errors.csv')
df_calib_error = pd.read_csv(csv_calib_error, sep=';')

csv_murphy = os.path.join(root_stat, '02_categorical', 'murphy_measures.csv')
df_murphy = pd.read_csv(csv_murphy, sep=';')

csv_failed_trials = os.path.join(root_stat, '04_failed_trials', 'failed_trials.csv')
df_failed_trials = pd.read_csv(csv_failed_trials, sep=';')


csv_cad = os.path.join(root_stat, '02_categorical', 'clinically_acceptable_difference.csv')
df_cad = pd.read_csv(csv_cad, sep=',')

list_identifier = sorted(df_val_trials['identifier'].tolist())

ignore_id_p = ['P11', 'P19']

In [106]:
def get_title_measure_name(measure):
    """returns a string based on the murphy measure for a figure_title"""
    match measure:
        case 'PeakVelocity_mms':
            title = 'Peak Endeffector Velocity'
        case 'elbowVelocity':
            title = 'Peak Elbow Velocity'
        case 'tTopeakV_s':
            title = 'Time to Peak Velocity'
        case 'tToFirstpeakV_s':
            title = 'Time to First Peak Velocity'
        case 'tTopeakV_rel':
            title = 'Relative time to Peak Velocity relative'
        case 'tToFirstpeakV_rel':
            title = 'Relative time to First Peak Velocity'
        case 'NumberMovementUnits':
            title = 'Number of Movement Units'
        case 'InterjointCoordination':
            title = 'Interjoint Coordination'
        case 'trunkDisplacementMM':
            title = 'Trunk Displacement'
        case 'trunkDisplacementDEG':
            title = 'Trunk Displacement'
        case 'ShoulderFlexionReaching':
            title = 'Shoulder Flexion Reaching'
        case 'ElbowExtension':
            title = 'Elbow Extension'
        case 'shoulderAbduction':
            title = 'Shoulder Abduction'
        case 'shoulderFlexionDrinking':
            title = 'Shoulder Flexion Drinking'
            title = 'Trunk Angle'
        case 'hand_vel':
            title = 'Hand Velocity'
        case 'elbow_vel':
            title = 'Elbow Velocity'
        case 'trunk_disp':
            title = 'Trunk Displacement'
        case 'trunk_ang':
            title = 'Trunk Angle'
        case 'elbow_flex_pos':
            title = 'Elbow Flexion'
        case 'shoulder_flex_pos':
            title = 'Shoulder Flexion'
        case 'shoulder_abduction_pos':
            title = 'Shoulder Abduction'
        case _:
            title = ''
    return title

def get_unit(kin):

    cases_deg = ['trunk_ang', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos',
                 'trunkDisplacementDEG', 'ShoulderFlexionReaching', 'ElbowExtension',
                 'shoulderAbduction', 'shoulderFlexionDrinking']

    match kin:
        case 'hand_vel' | 'PeakVelocity_mms':
            unit = 'mm/s'
        case 'elbow_vel' | 'elbowVelocity':
            unit = 'deg/s'
        case 'trunk_disp' | 'trunkDisplacementMM':
            unit = 'mm'
        case k if k in cases_deg:
            unit = 'deg'
        case 'tTopeakV_s' | 'tToFirstpeakV_s' :
            unit = 's'
        case 'tTopeakV_rel' | 'tToFirstpeakV_rel':
            unit = '%'
        case _:
            unit = ''

    return unit

def get_cad(df, measure):
    match measure:
        case 'PeakVelocity_mms' | 'hand_vel':
            measure_name = 'peak_V'
        case 'elbowVelocity' | 'elbow_vel':
            measure_name = 'peak_V_elb'
        case 'tTopeakV_s':
            measure_name = 't_to_PV'
        case 'tToFirstpeakV_s':
            measure_name = 't_first_PV'
        case 'tTopeakV_rel':
            measure_name = 't_PV_rel'
        case 'tToFirstpeakV_rel':
            measure_name = 't_first_PV_rel'
        case 'NumberMovementUnits':
            measure_name = 'n_mov_units'
        case 'InterjointCoordination':
            measure_name = 'interj_coord'
        case 'trunkDisplacementMM' | 'trunk_disp':
            measure_name = 'trunk_disp'
        case 'trunkDisplacementDEG' | 'trunk_ang':
            return None
        case 'ShoulderFlexionReaching' | 'elbow_flex_pos':
            measure_name = 'arm_flex_reach'
        case 'ElbowExtension' | 'shoulder_flex_pos':
            measure_name = 'elb_ext'
        case 'shoulderAbduction' | 'shoulder_abduction_pos':
            measure_name = 'arm_abd'
        case 'shoulderFlexionDrinking' | 'shoulder_flex_pos':
            measure_name = 'arm_flex_drink'
        case _:
            return

    return df.loc[0, measure_name]

## kinematic measures

In [107]:
df_corr_murphy = pd.read_csv(os.path.join(dir_stat_cat, 'stat_murphy_corr.csv'), sep=';')

if 'Unnamed: 0' in df_corr_murphy.columns:
    df_corr_murphy = df_corr_murphy.drop(columns=['Unnamed: 0'])

In [108]:
len_before = len(df_corr_murphy)
df_corr_murphy.dropna(inplace=True)
len_after = len(df_corr_murphy)
print(f'Number of rows dropped: {len_before - len_after}')

Number of rows dropped: 65


In [109]:
df_grouped = df_corr_murphy.groupby(['id_s', 'measure']).mean()

C:\Users\johan\AppData\Local\Temp\ipykernel_25076\2810033707.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [110]:
df_grouped.columns

Index(['pearson', 'pearson_p'], dtype='object')

In [111]:
df_plot = pd.DataFrame(columns = ['id_s', 'measure', 'correlation', 'p_value'])

for index in df_grouped.index:
    id_s = index[0]
    measure = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, measure, r, p]], columns = ['id_s', 'measure', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [112]:
df_plot

,id_s,measure,correlation,p_value
0,S001,ElbowExtension,0.857928,1.426104e-61
1,S001,InterjointCoordination,0.922508,6.074450e-85
2,S001,NumberMovementUnits,0.394871,6.095553e-10
3,S001,PeakVelocity_mms,0.792647,1.785234e-38
4,S001,ShoulderFlexionReaching,0.695428,1.007662e-25
...,...,...,...,...
242,S026,tToFirstpeakV_s,0.161560,4.197175e-01
243,S026,tTopeakV_rel,-0.054279,1.998167e-01
244,S026,tTopeakV_s,0.161560,4.197175e-01
245,S026,trunkDisplacementDEG,0.238558,4.554908e-02


In [113]:
df_plot['id_s'].unique() 

array(['S001', 'S002', 'S006', 'S007', 'S008', 'S011', 'S012', 'S013',
       'S014', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023',
       'S024', 'S025', 'S026'], dtype=object)

In [114]:
id_s_multi = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S011', 'S012', 'S013', 'S014',]
id_s_single = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023',
       'S024', 'S025', 'S026']

In [115]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
fig.add_hline(y=-0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'Pearson Correlation of all kinematic measures and settings',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_all.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_all.png')
fig.write_image(path, scale = 5)

In [116]:
id_s_reduced = ['S001', 'S002', 'S008', 'S014', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']

In [117]:
fig = px.bar(df_plot[df_plot['id_s'].isin(id_s_reduced)], x='id_s', y='correlation', color='measure', title=f'Pearson Correlation', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
             template = 'plotly', barmode="group")

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')

fig.update_layout(title= f'Pearson Correlation of chosen settings',
          xaxis_title=f'Settings',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

fig.show()

os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_reduced.html')
fig.write_html(path)

path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_reduced.png')
fig.write_image(path, scale = 5)

In [118]:
measures = df_plot['measure'].unique()

for measure in measures:
    df_plot_measure = df_plot[df_plot['measure'] == measure]
    title_name = get_title_measure_name(measure)
    
    fig = px.bar(df_plot_measure[df_plot_measure['id_s'].isin(id_s_reduced)], x='id_s', y='correlation', title=f'Pearson Correlation of {title_name}', labels={'correlation': 'Pearson Correlation', 'id_s': 'Subject ID', 'measure': 'Kin. Measure'},
                 template = 'plotly')
    
    fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'minimum correlation')
    
    fig.update_layout(title= f'Pearson Correlation of {title_name}',
              xaxis_title=f'Settings',
              yaxis_title=f'Correlation [r]',
                      width=1300, height=600
              )
    
    fig.show()
    os.makedirs(os.path.join(dir_plots_cat, '03_correlation'), exist_ok=True)
    
    path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_{measure}.html')
    fig.write_html(path)
    
    path = os.path.join(dir_plots_cat, '03_correlation', f'pearson_corr_{measure}.png')
    fig.write_image(path, scale = 5)

## Timeseries


In [119]:
csv_corr_ts = os.path.join(dir_stat_cont, '01_results', 'time_series_correlation.csv')
df_corr_ts = pd.read_csv(csv_corr_ts, sep=';')

In [120]:
df_corr_ts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127904 entries, 0 to 127903
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   id_s       127904 non-null  object 
 1   id_p       127904 non-null  object 
 2   id_t       127904 non-null  object 
 3   condition  127904 non-null  object 
 4   side       127904 non-null  object 
 5   dynamic    127904 non-null  object 
 6   metric     127904 non-null  object 
 7   pearson    127891 non-null  float64
 8   pearson_p  127891 non-null  float64
dtypes: float64(2), object(7)
memory usage: 8.8+ MB


In [121]:
df_corr_ts[df_corr_ts['dynamic'] == 'fixed']

,id_s,id_p,id_t,condition,side,dynamic,metric,pearson,pearson_p
0,S001,P07,T002,unaffected,L,fixed,hand_vel,0.991326,9.745339e-302
1,S001,P07,T002,unaffected,L,fixed,elbow_vel,0.982080,1.661938e-248
2,S001,P07,T002,unaffected,L,fixed,trunk_disp,0.992713,1.502667e-314
3,S001,P07,T002,unaffected,L,fixed,trunk_ang,0.717234,2.770792e-55
4,S001,P07,T002,unaffected,L,fixed,elbow_flex_pos,0.995176,0.000000e+00
...,...,...,...,...,...,...,...,...,...
63947,S026,P19,T094,affected,R,fixed,trunk_disp,0.920059,1.529540e-179
63948,S026,P19,T094,affected,R,fixed,trunk_ang,0.263348,2.209028e-08
63949,S026,P19,T094,affected,R,fixed,elbow_flex_pos,0.917893,4.068752e-177
63950,S026,P19,T094,affected,R,fixed,shoulder_flex_pos,0.965589,3.788099e-257


In [122]:
metrics_ts = ['hand_vel', 'elbow_vel', 'trunk_disp', 'elbow_flex_pos', 'shoulder_flex_pos', 'shoulder_abduction_pos']

df_corr_ts = df_corr_ts[df_corr_ts['metric'].isin(metrics_ts)]


In [123]:
df_corr_ts[df_corr_ts['dynamic'] == 'fixed'].groupby(['id_s', 'id_p', 'metric']).mean(numeric_only=True)

pearson      pearson_p
id_s id_p metric                                         
S001 P07  elbow_flex_pos          0.992834   0.000000e+00
          elbow_vel               0.959132  3.735755e-158
          hand_vel                0.984898  3.999281e-117
          shoulder_abduction_pos  0.962147  1.360787e-234
          shoulder_flex_pos       0.979282  8.751729e-261
...                                    ...            ...
S026 P19  elbow_vel               0.657499   2.925492e-16
          hand_vel                0.496067   5.338716e-04
          shoulder_abduction_pos  0.505887   4.462957e-02
          shoulder_flex_pos       0.954931  3.265986e-102
          trunk_disp              0.736900   1.627525e-04

[912 rows x 2 columns]

In [124]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_p',  'metric', 'correlation', 'p_value'])
df_grouped = df_corr_ts[df_corr_ts['dynamic'] == 'fixed'].groupby(['id_s', 'id_p', 'metric']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    metric = index[2]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_p, metric, r, p]], columns = ['id_s', 'id_p',  'metric', 'correlation', 'p_value'], index = [0])], ignore_index=True)

In [125]:
df_corr_ts[df_corr_ts['id_s'] == 'S001'].describe()

,pearson,pearson_p
count,8736.000000,8.736000e+03
mean,0.719058,2.503362e-02
std,0.381596,1.215799e-01
min,-0.793985,0.000000e+00
25%,0.536459,0.000000e+00
50%,0.939241,1.101771e-232
75%,0.984022,1.232667e-42
max,0.999677,9.988419e-01


In [126]:
s_multi = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010','S011', 'S012', 'S013', 'S014', 'S015', 'S016']
s_single = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']

df_plot = pd.DataFrame(columns = ['id_s', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(s_multi))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_p, r, p]], columns = ['id_s', 'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

### Multi-Cam

Let's first look at Multi-cam Settings

In [127]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for multi-cam settings',
          xaxis_title=f'Setting',
          yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

os.makedirs(os.path.join(dir_plots_cont, '03_correlation'), exist_ok=True)

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_multi_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_multi_cam.png')
fig.write_image(path, scale = 5)


Settings using less than five cameras show bad performance.

In [128]:
s_multi = ['S001', 'S002', 'S003', 'S004', 'S005', 'S006', 'S007', 'S008', 'S009', 'S010','S011', 'S012', 'S013', 'S014', 'S015', 'S016']
s_single = ['S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']

df_plot = pd.DataFrame(columns = ['id_s', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(s_single))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_p, r, p]], columns = ['id_s', 'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [129]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for single-cam settings',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
                  )

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_single_cam.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_single_cam.png')
fig.write_image(path)


In [130]:
idx_s_to_use = ['S001', 'S002',  'S008', 'S017', 'S018', 'S019', 'S020', 'S021', 'S022', 'S023', 'S024', 'S025', 'S026']

idx_p_to_use = ['P07', 'P10', 'P08', 'P12', 'P15', 'P241', 'P251', 'P242', 'P252']

In [131]:
df_plot['id_p'].unique()

array(['P07', 'P10', 'P08', 'P11', 'P12', 'P15', 'P19', 'P241', 'P251',
       'P242', 'P252'], dtype=object)

In [132]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'id_p']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    id_p = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, id_p, r, p]], columns = ['id_s', 'id_p', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [133]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='id_p', template = 'plotly', barmode='group', labels={"id_p": "Patient"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for chosen settings',
                  xaxis_title=f'Setting',
                  yaxis_title=f'Correlation [r]',
                  width=1300, height=600
          )

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_chosen_settings.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_chosen_settings.png')
fig.write_image(path, scale = 5)


In [134]:
df_corr_ts

,id_s,id_p,id_t,condition,side,dynamic,metric,pearson,pearson_p
0,S001,P07,T002,unaffected,L,fixed,hand_vel,0.991326,9.745339e-302
1,S001,P07,T002,unaffected,L,fixed,elbow_vel,0.982080,1.661938e-248
2,S001,P07,T002,unaffected,L,fixed,trunk_disp,0.992713,1.502667e-314
4,S001,P07,T002,unaffected,L,fixed,elbow_flex_pos,0.995176,0.000000e+00
5,S001,P07,T002,unaffected,L,fixed,shoulder_flex_pos,0.999248,0.000000e+00
...,...,...,...,...,...,...,...,...,...
127898,S026,P19,T094,affected,R,dynamic,elbow_vel,0.715334,2.128738e-69
127899,S026,P19,T094,affected,R,dynamic,trunk_disp,0.912271,5.871832e-170
127901,S026,P19,T094,affected,R,dynamic,elbow_flex_pos,0.931247,5.907622e-192
127902,S026,P19,T094,affected,R,dynamic,shoulder_flex_pos,0.974626,1.306540e-283


In [135]:
df_plot = pd.DataFrame(columns = ['id_s', 'id_p', 'correlation', 'p_value'])
df_grouped = df_corr_ts[(df_corr_ts['dynamic'] == 'fixed') & (df_corr_ts['id_s'].isin(idx_s_to_use)) & (df_corr_ts['id_p'].isin(idx_p_to_use))].groupby(['id_s', 'metric']).mean(numeric_only=True)

for index in df_grouped.index:
    id_s = index[0]
    metric = index[1]
    r = df_grouped.loc[index]['pearson']
    p = df_grouped.loc[index]['pearson_p']
    
    df_plot = pd.concat([df_plot, pd.DataFrame([[id_s, metric, r, p]], columns = ['id_s', 'metric', 'correlation', 'p_value'], index = [0])], ignore_index=True)
    df_plot.sort_values(by='id_s', inplace=True)

In [136]:
fig = px.bar(df_plot, x='id_s', y='correlation', color='metric', template = 'plotly', barmode='group', labels={"metric": "Kin. metrics"})

fig.add_hline(y=0.9, line_dash='dot', line_color='red', name = 'corr = 0.9')

fig.update_layout(title= f'Pearson Correlation for chosen settings and kinematic metrics',
                  yaxis_title=f'Mean Correlation [r]',
                  width=1300, height=600
                  )

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_kinematics.html')
fig.write_html(path)

path = os.path.join(dir_plots_cont, '03_correlation', 'pearson_corr_kinematics.png')
fig.write_image(path, scale = 5)